# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dalandzadgad,43.5708,104.4250,6.45,33,23,2.70,MN,1698440793
1,1,north shore,-36.8000,174.7500,13.04,62,100,2.24,NZ,1698440793
2,2,enewetak,11.3474,162.3373,28.43,74,47,5.16,MH,1698440793
3,3,puerto natales,-51.7236,-72.4875,4.60,73,18,8.39,CL,1698440499
4,4,kingston,17.9970,-76.7936,28.53,82,40,7.20,JM,1698440080


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.2,
    color = "City",
)

# Display the map
city_map

c:\Users\mdermer\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
#Narrow down cities that fit criteria and drop any results with null values
                                                                  
narrowed_city_df=city_data_df
narrowed_city_df= narrowed_city_df.loc[(narrowed_city_df["Max Temp"] > 21) & (narrowed_city_df["Max Temp"] < 27) & (narrowed_city_df["Cloudiness"] == 0)& (narrowed_city_df["Wind Speed"] >4.5)]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']

# Display sample data
narrowed_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,145,dakhla,23.6848,-15.9580,21.09,85,0,7.57,EH,1698440824
1,187,punta del este,-34.9667,-54.9500,24.06,74,0,5.36,UY,1698440833
2,361,port hueneme,34.1478,-119.1951,26.97,62,0,5.66,US,1698440870
3,512,inhambane,-23.8650,35.3833,22.50,75,0,5.09,MZ,1698440904


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=narrowed_city_df
del hotel_df['City_ID']
del hotel_df["Max Temp"]
del hotel_df["Cloudiness"]
del hotel_df["Wind Speed"]
del hotel_df["Date"]




# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Country
0,dakhla,23.6848,-15.9580,85,EH
1,punta del este,-34.9667,-54.9500,74,UY
2,port hueneme,34.1478,-119.1951,62,US
3,inhambane,-23.8650,35.3833,75,MZ


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
hotellist = []
# Set parameters to search for a hotel
radius=10000
params = {
        "categories":"accommodation.hotel",
        "apiKey": geoapify_key
    }
  
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for i, row in hotel_df.iterrows():
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address =  name_address.json()
    

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[i, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except Exception as E:

        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[i, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[i, 'City']} - nearest hotel: {hotel_df.loc[i, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
dakhla - nearest hotel: Hotel El Massira
punta del este - nearest hotel: Hotel Aquarium
port hueneme - nearest hotel: No hotel found
inhambane - nearest hotel: Hotel de Inhambane


,City,Lat,Lng,Humidity,Country,Hotel Name
0,dakhla,23.6848,-15.9580,85,EH,Hotel El Massira
1,punta del este,-34.9667,-54.9500,74,UY,Hotel Aquarium
2,port hueneme,34.1478,-119.1951,62,US,No hotel found
3,inhambane,-23.8650,35.3833,75,MZ,Hotel de Inhambane


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
# Configure the map plot

hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.2,
    color = "City",
    hover_cols=["Country","Hotel Name"]
)



# Display the map
hotel_map 


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)